In [1]:
import pandas as pd
import vertica_python
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import os

In [2]:
username = os.getenv('VERTICA_USERNAME')
password = os.getenv('VERTICA_PASSWORD')
conn_info = {'host': 'warehouse.analytics.edx.org',
             'port': 5433,
             'user': username,
             'password': password,
             'database': 'warehouse',
             'read_timeout': 600,
             'unicode_error': 'strict',
             'ssl': False}

# Queries
* * *
### Overall Completion Rate

In [6]:
#DROP TABLE IF EXISTS ahemphill.completion_rate;
#CREATE TABLE ahemphill.completion_rate AS
#	SELECT
#		a.user_id,
#		a.course_id,
#		a.current_enrollment_mode,
#		a.first_enrollment_mode,
#		a.first_enrollment_time,
#		b.catalog_course_title,
#		b.end_time AS end_time_from_d_course,
#		c.enddate AS end_time_ed_services_table,
#		d.certificate_mode,
#		d.has_passed,
#		CASE WHEN b.end_time BETWEEN '2016-04-01' AND '2016-07-01' THEN 'Q4_2016' ELSE 'Q1_2017' END AS quarter_d_course,
#		CASE WHEN c.enddate BETWEEN '2016-04-01' AND '2016-07-01' THEN 'Q4_2016' ELSE 'Q1_2017' END AS quarter_ed_services
#	FROM
#		production.d_user_course a	
#	LEFT JOIN
#		production.d_course b
#	ON
#	       a.course_id = b.course_id
#	LEFT JOIN
#	       ed_services.CourseCatalog_20161010 c
#	ON
#	       a.course_id = c.CourseID
#	LEFT JOIN
#	    production.d_user_course_certificate d
#	ON     
#	    a.user_id = d.user_id
#	    AND a.course_id = d.course_id#
#	WHERE
#		b.pacing_type = 'instructor_paced'
#		AND c.enddate BETWEEN '2016-04-01' AND '2016-10-01'
#		AND a.first_enrollment_time <= c.enddate
#		AND a.first_enrollment_time IS NOT NULL


In [58]:
from datetime import date
from dateutil.rrule import rrule, DAILY
a = date(2016, 6, 1)
b = date(2017, 2, 11)

In [66]:
master_df = pd.DataFrame()
connection = vertica_python.connect(**conn_info)

for dt in rrule(DAILY, dtstart=a, until=b):

    engagement_activity_query = """
    
    DROP TABLE IF EXISTS ahemphill.engagement_activity_loop2;
    CREATE TABLE IF NOT EXISTS ahemphill.engagement_activity_loop2 AS

    SELECT
        CAST({date} AS DATE) AS date,
        a.user_id,
        SUM(CASE WHEN b.date >= CAST({date} AS DATE) - 7 THEN 1 ELSE 0 END) AS active_7d,
        SUM(CASE WHEN b.date >= CAST({date} AS DATE) - 14 THEN 1 ELSE 0 END) AS active_14d,
        SUM(CASE WHEN b.date >= CAST({date} AS DATE) - 30 THEN 1 ELSE 0 END) AS active_30d,
        SUM(CASE WHEN b.date >= CAST({date} AS DATE) - 7 AND b.activity_type IN ('ATTEMPTED_PROBLEM', 'PLAYED_VIDEO') THEN 1 ELSE 0 END) AS engaged_7d,
        SUM(CASE WHEN b.date >= CAST({date} AS DATE) - 14 AND b.activity_type IN ('ATTEMPTED_PROBLEM', 'PLAYED_VIDEO') THEN 1 ELSE 0 END) AS engaged_14d,
        SUM(CASE WHEN b.date >= CAST({date} AS DATE) - 30 AND b.activity_type IN ('ATTEMPTED_PROBLEM', 'PLAYED_VIDEO') THEN 1 ELSE 0 END) AS engaged_30d

     FROM 
     (
        SELECT 
            a.user_id, 
            a.course_id
        FROM 
            production.d_user_course a 
        JOIN 
            production.d_course b 
        ON 
            a.course_id = b.course_id 
        AND
            first_enrollment_time <= CAST({date} AS DATE)
        AND 
            CAST({date} AS DATE) BETWEEN b.start_time AND b.end_time
        AND 
            datediff('day',first_enrollment_time, CAST({date} AS DATE)) <= 90
        LEFT JOIN 
            production.d_user_course_certificate c
        ON 
            a.user_id = c.user_id
        AND 
            a.course_id = c.course_id
        AND
            c.modified_date <= CAST({date} AS DATE)
        WHERE 
            (
            has_passed = 0 
            OR has_passed IS NULL
            )
            AND
            (
            a.last_unenrollment_time is null
            OR
            a.last_unenrollment_time >= CAST({date} AS DATE)
            )
    ) a
    LEFT JOIN 
        production.f_user_activity b
    ON 
        a.user_id = b.user_id
    AND 
        a.course_id = b.course_id
    AND b.date <= CAST({date} AS DATE)
    GROUP BY
        1,2;
    
    
    SELECT 
    {date},
    count(1) AS eligible_users,
    SUM(CASE WHEN active_7d > 0 THEN 1 ELSE 0 END) * 100.0/count(1) AS active_rate_7d,
    SUM(CASE WHEN active_14d > 0 THEN 1 ELSE 0 END) * 100.0/count(1) AS active_rate_14d,
    SUM(CASE WHEN active_30d > 0 THEN 1 ELSE 0 END) * 100.0/count(1) AS active_rate_30d,
    SUM(CASE WHEN engaged_7d > 0 THEN 1 ELSE 0 END) * 100.0/count(1) AS engagement_rate_7d,
    SUM(CASE WHEN engaged_14d > 0 THEN 1 ELSE 0 END) * 100.0/count(1) AS engagement_rate_14d,
    SUM(CASE WHEN engaged_30d > 0 THEN 1 ELSE 0 END) * 100.0/count(1) AS engagement_rate_30d

    FROM ahemphill.engagement_activity_loop2
    GROUP BY 1
    """.format(date=dt.strftime("\'%Y-%m-%d\'"))
    
    cur = connection.cursor('dict')
    cur.execute(engagement_activity_query)
    extract = cur.fetchall()
    engagement_activity_df = pd.DataFrame(extract)
    master_df = master_df.append(engagement_activity_df)
    print dt
    
connection.close()

2016-06-01 00:00:00
2016-06-02 00:00:00
2016-06-03 00:00:00
2016-06-04 00:00:00
2016-06-05 00:00:00
2016-06-06 00:00:00
2016-06-07 00:00:00
2016-06-08 00:00:00
2016-06-09 00:00:00
2016-06-10 00:00:00
2016-06-11 00:00:00
2016-06-12 00:00:00
2016-06-13 00:00:00
2016-06-14 00:00:00
2016-06-15 00:00:00
2016-06-16 00:00:00
2016-06-17 00:00:00
2016-06-18 00:00:00
2016-06-19 00:00:00
2016-06-20 00:00:00
2016-06-21 00:00:00
2016-06-22 00:00:00
2016-06-23 00:00:00
2016-06-24 00:00:00
2016-06-25 00:00:00
2016-06-26 00:00:00
2016-06-27 00:00:00
2016-06-28 00:00:00
2016-06-29 00:00:00
2016-06-30 00:00:00
2016-07-01 00:00:00
2016-07-02 00:00:00
2016-07-03 00:00:00
2016-07-04 00:00:00
2016-07-05 00:00:00
2016-07-06 00:00:00
2016-07-07 00:00:00
2016-07-08 00:00:00
2016-07-09 00:00:00
2016-07-10 00:00:00
2016-07-11 00:00:00
2016-07-12 00:00:00
2016-07-13 00:00:00
2016-07-14 00:00:00
2016-07-15 00:00:00
2016-07-16 00:00:00
2016-07-17 00:00:00
2016-07-18 00:00:00
2016-07-19 00:00:00
2016-07-20 00:00:00


In [67]:
print 'done'

done


In [ ]:
#engagement_activity_df

In [68]:
master_df

,?column?,active_rate_14d,active_rate_30d,active_rate_7d,eligible_users,engagement_rate_14d,engagement_rate_30d,engagement_rate_7d
0,2016-06-01,27.6091938369903366705,41.7621706938911281996,19.1176979464448167740,895861,18.0231084956259955506,27.5350751958172082499,12.2757883198397965756
0,2016-06-02,27.6228312679765518101,41.3914468667366933241,19.4613961940726515830,931074,17.9628042454198055149,27.2149152484120488812,12.5399270090239873522
0,2016-06-03,27.8193117481493322043,41.5102572466037788833,19.4900646086650350693,925882,18.0484122166755590885,27.2861984572548121683,12.5732004726304215872
0,2016-06-04,27.8180956280031890366,41.3036264517658212847,19.3324570665172703570,928180,18.0759119998276196427,27.2105626063910017454,12.5360382684393113405
0,2016-06-05,27.9503045486068178031,41.2313117900361802670,19.5747609038077983797,930065,18.1871159542612613097,27.2000344061974163096,12.7040583185046206448
0,2016-06-06,28.1227202503199124141,41.2489685067021243054,20.0996321958548918604,919783,18.3249744776757126409,27.2792604342546013571,13.0263333851571511976
0,2016-06-07,28.6547526421398759752,41.8805594507757913692,20.6535706103511896132,903529,18.6992337822028955352,27.7374605574364519567,13.3722326566164450726
0,2016-06-08,29.0865014292805669194,42.1606742438506701357,21.3144674004210814604,934736,19.0369259341675082590,27.8987863953030588316,13.8659471765289878640
0,2016-06-09,29.1433023457954822502,42.3853337432027051379,21.0046746308525636269,943390,19.1756325591748905543,28.0899733938243992410,13.7309066239837182925
0,2016-06-10,29.1633960274807644157,42.4474910543836425430,20.6348720039976630997,946553,19.2685459768232734987,28.1612334438747751050,13.4857741721805329443


In [69]:
master_df.to_csv('engagement_activity_metric_draft.csv')

In [82]:
a = date(2016, 6, 1)
b = date(2017, 2, 11)

In [84]:
master_weekly_df = pd.DataFrame()
connection = vertica_python.connect(**conn_info)

for dt in rrule(DAILY, dtstart=a, until=b):

    engagement_activity_weekly_query = """
    
    DROP TABLE IF EXISTS ahemphill.engagement_activity_weekly;
    CREATE TABLE IF NOT EXISTS ahemphill.engagement_activity_weekly AS

    SELECT
        CAST({date} AS DATE) AS date,
        a.user_id,
        a.course_id,
        a.start_time,
        a.end_time,
        datediff('day', first_enrollment_time, CAST({date} AS DATE)) AS date_in_course,
        SUM(CASE WHEN 
        datediff('day', first_enrollment_time, b.date) BETWEEN 0 AND 7 THEN 1 ELSE 0 END) AS active_wk1,
        SUM(CASE WHEN 
        datediff('day', first_enrollment_time, b.date) BETWEEN 8 AND 14 THEN 1 ELSE 0 END) AS active_wk2,
        SUM(CASE WHEN 
        datediff('day', first_enrollment_time, b.date) BETWEEN 15 AND 21 THEN 1 ELSE 0 END) AS active_wk3,
        SUM(CASE WHEN 
        datediff('day', first_enrollment_time, b.date) BETWEEN 22 AND 28 THEN 1 ELSE 0 END) AS active_wk4,
        SUM(CASE WHEN 
        datediff('day', first_enrollment_time, b.date) BETWEEN 0 AND 7 AND b.activity_type IN ('ATTEMPTED_PROBLEM', 'PLAYED_VIDEO') THEN 1 ELSE 0 END) AS engaged_wk1,
        SUM(CASE WHEN 
        datediff('day', first_enrollment_time, b.date) BETWEEN 8 AND 14 AND b.activity_type IN ('ATTEMPTED_PROBLEM', 'PLAYED_VIDEO') THEN 1 ELSE 0 END) AS engaged_wk2,
        SUM(CASE WHEN 
        datediff('day', first_enrollment_time, b.date) BETWEEN 15 AND 21 AND b.activity_type IN ('ATTEMPTED_PROBLEM', 'PLAYED_VIDEO') THEN 1 ELSE 0 END) AS engaged_wk3,
        SUM(CASE WHEN 
        datediff('day', first_enrollment_time, b.date) BETWEEN 22 AND 28 AND b.activity_type IN ('ATTEMPTED_PROBLEM', 'PLAYED_VIDEO') THEN 1 ELSE 0 END) AS engaged_wk4

     FROM 
     (
        SELECT 
            a.user_id, 
            a.course_id,
            a.first_enrollment_time,
            b.start_time,
            b.end_time
        FROM 
            production.d_user_course a 
        JOIN 
            production.d_course b 
        ON 
            a.course_id = b.course_id 
        AND 
            CAST({date} AS DATE) BETWEEN b.start_time AND b.end_time
        AND 
            datediff('day',first_enrollment_time, CAST({date} AS DATE)) BETWEEN 0 and  90
        LEFT JOIN 
            production.d_user_course_certificate c
        ON 
            a.user_id = c.user_id
        AND 
            a.course_id = c.course_id
        AND
            c.modified_date <= CAST({date} AS DATE)
        WHERE 
            (
                has_passed = 0 
                OR has_passed IS NULL
            )
            AND
            (
                a.last_unenrollment_time is null
                OR
                a.last_unenrollment_time >= CAST({date} AS DATE)
            )
    ) a
    LEFT JOIN 
        production.f_user_activity b
    ON 
        a.user_id = b.user_id
    AND 
        a.course_id = b.course_id
    AND b.date <= CAST({date} AS DATE)
    GROUP BY
        1,2,3,4,5,6;
    
    
SELECT
    date,
    course_id,
    sum(case when date_in_course between 0 and 7 then 1 else 0 end) as cnt_week1,
    sum(case when date_in_course between 8 and 14 then 1 else 0 end) as cnt_week2,
    sum(case when date_in_course between 15 and 21 then 1 else 0 end) as cnt_week3,
    sum(case when date_in_course between 22 and 28 then 1 else 0 end) as cnt_week4,
    sum(case when date_in_course between 0 and 7 AND active_wk1 > 0 then 1 else 0 end) as active_week1,
    sum(case when date_in_course between 8 and 14 AND active_wk2 > 0 then 1 else 0 end) as active_week2,
    sum(case when date_in_course between 15 and 21 AND active_wk3 > 0 then 1 else 0 end) as active_week3,
    sum(case when date_in_course between 22 and 28 AND active_wk4 > 0 then 1 else 0 end) as active_week4,
    sum(case when date_in_course between 0 and 7 AND engaged_wk1 > 0 then 1 else 0 end) as active_week1,
    sum(case when date_in_course between 8 and 14 AND engaged_wk2 > 0 then 1 else 0 end) as engaged_week2,
    sum(case when date_in_course between 15 and 21 AND engaged_wk3 > 0 then 1 else 0 end) as engaged_week3,
    sum(case when date_in_course between 22 and 28 AND engaged_wk4 > 0 then 1 else 0 end) as engaged_week4,


    sum(case when date_in_course between 0 and 7 then 1 else 0 end) as cnt_week1_building,
    sum(case when date_in_course between 8 and 14 AND active_wk1 > 0 then 1 else 0 end) as cnt_week2_building,
    sum(case when date_in_course between 15 and 21 AND active_wk1 > 0 AND active_wk2 > 0 then 1 else 0 end) as cnt_week3_building,
    sum(case when date_in_course between 22 and 28 AND active_wk1 > 0 AND active_wk2 > 0 AND active_wk3 > 0 then 1 else 0 end) as cnt_week4,
    sum(case when date_in_course between 0 and 7 AND active_wk1 > 0 then 1 else 0 end) as active_week1_building,
    sum(case when date_in_course between 8 and 14 AND active_wk1 >0 AND active_wk2 > 0 then 1 else 0 end) as active_week2_building,
    sum(case when date_in_course between 15 and 21 AND active_wk1 >0 AND active_wk2 > 0  AND active_wk3 > 0 then 1 else 0 end) as active_week3_building,
    sum(case when date_in_course between 22 and 28 AND active_wk1 >0 AND active_wk2 > 0  AND active_wk3 > 0 AND active_wk4 > 0 then 1 else 0 end) as active_week4_building,
    sum(case when date_in_course between 0 and 7 AND engaged_wk1 > 0 then 1 else 0 end) as engaged_week1_building,
    sum(case when date_in_course between 8 and 14 AND engaged_wk1 > 0 AND engaged_wk2 > 0 then 1 else 0 end) as engaged_week2_building,
    sum(case when date_in_course between 15 and 21 AND engaged_wk1 > 0 AND engaged_wk2 > 0 AND engaged_wk3 > 0 then 1 else 0 end) as engaged_week3_building,
    sum(case when date_in_course between 22 and 28 AND engaged_wk1 > 0 AND engaged_wk2 > 0 AND engaged_wk3 > 0 AND engaged_wk4 > 0 then 1 else 0 end) as engaged_week4_building
FROM
    ahemphill.engagement_activity_weekly
GROUP BY 1,2;
    """.format(date=dt.strftime("\'%Y-%m-%d\'"))
    
    cur = connection.cursor('dict')
    cur.execute(engagement_activity_weekly_query)
    extract = cur.fetchall()
    engagement_activity_weekly_df = pd.DataFrame(extract)
    master_weekly_df = master_weekly_df.append(engagement_activity_weekly_df)
    print dt
    
connection.close()

2016-06-01 00:00:00
2016-06-02 00:00:00
2016-06-03 00:00:00
2016-06-04 00:00:00
2016-06-05 00:00:00
2016-06-06 00:00:00
2016-06-07 00:00:00
2016-06-08 00:00:00
2016-06-09 00:00:00
2016-06-10 00:00:00
2016-06-11 00:00:00
2016-06-12 00:00:00
2016-06-13 00:00:00
2016-06-14 00:00:00
2016-06-15 00:00:00
2016-06-16 00:00:00
2016-06-17 00:00:00
2016-06-18 00:00:00
2016-06-19 00:00:00
2016-06-20 00:00:00
2016-06-21 00:00:00
2016-06-22 00:00:00
2016-06-23 00:00:00
2016-06-24 00:00:00
2016-06-25 00:00:00
2016-06-26 00:00:00
2016-06-27 00:00:00
2016-06-28 00:00:00
2016-06-29 00:00:00
2016-06-30 00:00:00
2016-07-01 00:00:00
2016-07-02 00:00:00
2016-07-03 00:00:00
2016-07-04 00:00:00
2016-07-05 00:00:00
2016-07-06 00:00:00
2016-07-07 00:00:00
2016-07-08 00:00:00
2016-07-09 00:00:00
2016-07-10 00:00:00
2016-07-11 00:00:00
2016-07-12 00:00:00
2016-07-13 00:00:00
2016-07-14 00:00:00
2016-07-15 00:00:00
2016-07-16 00:00:00
2016-07-17 00:00:00
2016-07-18 00:00:00
2016-07-19 00:00:00
2016-07-20 00:00:00


In [91]:
len(master_weekly_df)#[master_weekly_df.course_id == 'course-v1:UC3Mx+IT.1.1x+3T2015']

110622

In [103]:
#master_weekly_df.dtypes
master_weekly_df['date'] = master_weekly_df['date'].astype('str') 
master_weekly_df.to_csv('engagement_activity_weekly_metric_draft.csv')

In [96]:
master_weekly_df#[master_weekly_df['date']=='2016-06-01']

,active_week1,active_week1_building,active_week2,active_week2_building,active_week3,active_week3_building,active_week4,active_week4_building,cnt_week1,cnt_week1_building,...,cnt_week4,course_id,date,engaged_week1_building,engaged_week2,engaged_week2_building,engaged_week3,engaged_week3_building,engaged_week4,engaged_week4_building
0,24,48,1,1,3,0,0,0,149,149,...,0,course-v1:SaveALifeX+PALS101x+3T2015,2016-06-01,24,0,0,2,0,0,0
1,63,98,14,11,8,4,5,2,172,172,...,8,course-v1:Microsoft+DAT202.1x+2T2016,2016-06-01,63,6,6,2,1,2,0
2,0,0,0,0,0,0,0,0,0,0,...,0,course-v1:WellesleyX+SOC101x+2T2016,2016-06-01,0,0,0,0,0,0,0
3,48,83,12,12,10,6,23,14,108,108,...,19,course-v1:IDBx+IDB5x+2016_T3,2016-06-01,48,11,10,6,2,17,5
4,112,196,16,15,4,1,10,1,296,296,...,2,course-v1:DavidsonNext+Phy_APccx+2T2015,2016-06-01,112,7,6,1,0,5,0
5,2,7,0,0,0,0,0,0,9,9,...,0,course-v1:Microsoft+CLD209.2x+2T2016,2016-06-01,2,0,0,0,0,0,0
6,2,4,0,0,0,0,1,0,4,4,...,0,course-v1:Microsoft+CLD208.2x+1T2016,2016-06-01,2,0,0,0,0,1,0
7,98,188,22,19,10,7,28,18,304,304,...,30,course-v1:Microsoft+CLD208.1x+1T2016,2016-06-01,98,14,9,6,2,10,2
8,145,259,34,29,23,15,14,5,389,389,...,8,course-v1:DelftX+AE1110x+2T2015,2016-06-01,145,18,15,12,9,3,3
9,0,0,0,0,1,0,1,0,0,0,...,0,course-v1:ASUx+DOGx002+1T2016,2016-06-01,0,0,0,0,0,1,0
